# US LNG Export Optimization & Route Arbitrage Model (Python | Linear Programming)

3. Shipping Cost Estimates
Approx. or synthetic shipping costs from U.S. Gulf to Japan, UK, India, etc.

Use: Cost input in objective function
🔧 If you can’t find real freight index data, use simplified assumptions (e.g., $2/MMBtu to Europe, $3 to Asia).

4. Max Capacity Estimates per Route (Synthetic OK)
Like: US Gulf to Japan: max 60 KT/month

Use to build LP constraints
🔧 Based on historical max exports per route — you can derive this from your export data.

5. Receiving Country Demand (Synthetic OK)
Monthly or seasonal demand for UK, Japan, India

Use: Constraints for how much volume they can realistically absorb
🔧 Can be synthetic or based on realistic assumptions like: “Japan imports ~80 KT/month.”



In [61]:
import pandas as pd

# Read Excel file
us_lng_export = pd.read_excel('/kaggle/input/lng-flow-optimization/US LNG Export.xls')
us_lng_export_price = pd.read_excel('/kaggle/input/lng-flow-optimization/US LNG Export Prices.xlsx')
hh_prices = pd.read_csv('/kaggle/input/lng-flow-optimization/Henry_Hub_Natural_Gas_Spot_Price.csv')

# Show the first 5 rows
#print(df.head())

In [62]:
# Key destinations
important_countries = ['United Kingdom', 'Japan', 'India', 'Spain', 'France', 'South Korea']

# Filter and reshape
volume_filtered = us_lng_export[['Date'] + [col for col in us_lng_export.columns if any(c in col for c in important_countries)]]
volume_long = volume_filtered.melt(id_vars='Date', var_name='Country', value_name='Volume_MMcf')
volume_long['Country'] = volume_long['Country'].str.extract(r'to (.+?) \(')
volume_long['Volume_MMcf'] = volume_long['Volume_MMcf'].fillna(0)


In [63]:
# Price processing
price_filtered = us_lng_export_price[['Date'] + [col for col in us_lng_export_price.columns if any(c in col for c in important_countries)]]
price_long = price_filtered.melt(id_vars='Date', var_name='Country', value_name='Price_per_Mcf')
price_long['Country'] = price_long['Country'].str.extract(r'to (.+?) \(')

# Merge
merged = pd.merge(volume_long, price_long, on=['Date', 'Country'], how='inner')

# Add metrics
merged['Revenue'] = merged['Volume_MMcf'] * merged['Price_per_Mcf']
merged['Volume_KT'] = merged['Volume_MMcf'] * 0.0198  # Approx conversion
merged['Price_per_KT'] = merged['Price_per_Mcf'] * 52.2

# Filter valid rows
filtered = merged[(merged['Volume_MMcf'] > 0) & (merged['Price_per_Mcf'].notna())]


In [64]:
# Aggregate by destination
country_prices = filtered.groupby('Country').agg({
    'Volume_KT': 'mean',
    'Price_per_KT': 'mean'
}).reset_index()
country_prices.columns = ['Destination', 'Avg_Volume_KT', 'Avg_Price_per_KT']

# Define base route assumptions
routes = pd.DataFrame({
    'Origin': ['US Gulf'] * 6,
    'Destination': ['United Kingdom', 'Japan', 'India', 'Spain', 'France', 'South Korea'],
    'Shipping_Cost_per_KT': [35, 80, 65, 40, 45, 78],
    'Max_Capacity_KT': [100, 100, 80, 60, 70, 60]
})

# Merge real prices
routes = pd.merge(routes, country_prices, on='Destination', how='inner')
routes['Profit_per_KT'] = routes['Avg_Price_per_KT'] - routes['Shipping_Cost_per_KT']
routes['Route_Key'] = routes['Origin'] + "_to_" + routes['Destination']


In [65]:
# Initialize LP model
model = LpProblem("US_LNG_Profit_Maximization", LpMaximize)

# Create decision variables
route_vars = {
    row['Route_Key']: LpVariable(row['Route_Key'], lowBound=0)
    for _, row in routes.iterrows()
}

# Objective: Maximize total profit
model += lpSum([
    route_vars[row['Route_Key']] * row['Profit_per_KT']
    for _, row in routes.iterrows()
]), "Total_Profit"

# Capacity constraints
for _, row in routes.iterrows():
    model += route_vars[row['Route_Key']] <= row['Max_Capacity_KT'], f"Cap_{row['Route_Key']}"

# US Gulf total export constraint
model += lpSum([route_vars[r] for r in route_vars]) <= 220, "US_Total_Supply"

# Solve
model.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.11/dist-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/5fdfa414fcfc45b4aede8301c48c8bd4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/5fdfa414fcfc45b4aede8301c48c8bd4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 31 RHS
At line 39 BOUNDS
At line 40 ENDATA
Problem MODEL has 7 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 6 (0) columns and 6 (-6) elements
0  Obj -0 Dual inf 1984.3891 (6)
1  Obj 79801.26
Optimal - objective value 79801.26
After Postsolve, objective 79801.26, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 79801.26019 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds

1

In [66]:
print("Status:", LpStatus[model.status])
print("\nOptimal Shipment Plan:\n")
for key, var in route_vars.items():
    print(f"{key}: {var.varValue:.1f} KT")

print(f"\nTotal Profit: ${value(model.objective):,.2f}")


Status: Optimal

Optimal Shipment Plan:

US Gulf_to_United Kingdom: 100.0 KT
US Gulf_to_Japan: 0.0 KT
US Gulf_to_India: 0.0 KT
US Gulf_to_Spain: 50.0 KT
US Gulf_to_France: 70.0 KT
US Gulf_to_South Korea: 0.0 KT

Total Profit: $79,801.26


In [67]:
# final optimized results
routes_data = {
    'Route_Key': [
        'US Gulf_to_United Kingdom',
        'US Gulf_to_Japan',
        'US Gulf_to_India',
        'US Gulf_to_Spain',
        'US Gulf_to_France',
        'US Gulf_to_South Korea'
    ],
    'Volume_KT': [100.0, 0.0, 0.0, 50.0, 70.0, 0.0],
    'Profit_per_KT': [34.59, -79.61, -64.64, 39.61, 44.60, -77.62],
    'Shipping_Cost_per_KT': [35, 80, 65, 50, 55, 78],
    'Selling_Price_per_KT': [69.59, 0, 0, 89.61, 99.60, 0]  # Profit + Cost
}

# Create DataFrame
route_report = pd.DataFrame(routes_data)

# Calculate Revenue, Cost, and Profit
route_report['Revenue'] = route_report['Volume_KT'] * route_report['Selling_Price_per_KT']
route_report['Cost'] = route_report['Volume_KT'] * route_report['Shipping_Cost_per_KT']
route_report['Profit'] = route_report['Revenue'] - route_report['Cost']

# Total profit and share
total_profit = route_report['Profit'].sum()
route_report['Profit_Share_%'] = (route_report['Profit'] / total_profit * 100).round(2)

# Sort and display
route_report = route_report.sort_values(by='Profit', ascending=False)
print(route_report[['Route_Key', 'Volume_KT', 'Revenue', 'Cost', 'Profit', 'Profit_Share_%']])



                   Route_Key  Volume_KT  Revenue    Cost  Profit  \
0  US Gulf_to_United Kingdom      100.0   6959.0  3500.0  3459.0   
4          US Gulf_to_France       70.0   6972.0  3850.0  3122.0   
3           US Gulf_to_Spain       50.0   4480.5  2500.0  1980.5   
1           US Gulf_to_Japan        0.0      0.0     0.0     0.0   
2           US Gulf_to_India        0.0      0.0     0.0     0.0   
5     US Gulf_to_South Korea        0.0      0.0     0.0     0.0   

   Profit_Share_%  
0           40.40  
4           36.47  
3           23.13  
1            0.00  
2            0.00  
5            0.00  


# Scenario Analysis


# Scenario 1 - UK Price Drops.

In [68]:
from pulp import *

# 1. Copy base routes to simulate scenario
scenario_routes = routes.copy()

# 2. Simulate price drop for UK
scenario_routes.loc[scenario_routes['Destination'] == 'United Kingdom', 'Avg_Price_per_KT'] = 30  # drop price aggressively

# 3. Recalculate Profit per KT
scenario_routes['Profit_per_KT'] = scenario_routes['Avg_Price_per_KT'] - scenario_routes['Shipping_Cost_per_KT']
scenario_routes['Route_Key'] = scenario_routes['Origin'] + "_to_" + scenario_routes['Destination']

# 4. Set up optimization model
model = LpProblem("US_LNG_Profit_Maximization_Scenario1", LpMaximize)

route_vars = {
    row['Route_Key']: LpVariable(row['Route_Key'], lowBound=0)
    for _, row in scenario_routes.iterrows()
}

# Objective
model += lpSum([
    route_vars[row['Route_Key']] * row['Profit_per_KT']
    for _, row in scenario_routes.iterrows()
]), "Total_Profit"

# Constraints
for _, row in scenario_routes.iterrows():
    model += route_vars[row['Route_Key']] <= row['Max_Capacity_KT'], f"Cap_{row['Route_Key']}"

model += lpSum([
    route_vars[r] for r in route_vars
]) <= 220, "US_Total_Supply"

# 5. Solve
model.solve()

# 6. Print Results
print("Status:", LpStatus[model.status])
print("\nOptimal Shipment Plan (Scenario 1: UK Price Drops):\n")
for key, var in route_vars.items():
    print(f"{key}: {var.varValue:.1f} KT")

print(f"\nTotal Profit: ${value(model.objective):,.2f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.11/dist-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/f68f2ff4fc694d0381a1e22032afc7be-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f68f2ff4fc694d0381a1e22032afc7be-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 31 RHS
At line 39 BOUNDS
At line 40 ENDATA
Problem MODEL has 7 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 5 (-1) columns and 5 (-7) elements
0  Obj -0 Dual inf 1609.8914 (5)
1  Obj 73872.592
Optimal - objective value 73872.592
After Postsolve, objective 73872.592, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 73872.59185 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock sec

# #  Scenario 2: France Increases Import Capacity

In [69]:
from pulp import *

# 1. Copy routes for scenario
scenario_routes = routes.copy()

# 2. Increase France's capacity
scenario_routes.loc[scenario_routes['Destination'] == 'France', 'Max_Capacity_KT'] = 120

# 3. Recalculate profit just to be sure (may not change, but good practice)
scenario_routes['Profit_per_KT'] = scenario_routes['Avg_Price_per_KT'] - scenario_routes['Shipping_Cost_per_KT']
scenario_routes['Route_Key'] = scenario_routes['Origin'] + "_to_" + scenario_routes['Destination']

# 4. Set up model
model = LpProblem("Scenario2_France_Capacity_Boost", LpMaximize)

route_vars = {
    row['Route_Key']: LpVariable(row['Route_Key'], lowBound=0)
    for _, row in scenario_routes.iterrows()
}

# Objective
model += lpSum([
    route_vars[row['Route_Key']] * row['Profit_per_KT']
    for _, row in scenario_routes.iterrows()
]), "Total_Profit"

# Constraints: route-level caps
for _, row in scenario_routes.iterrows():
    model += route_vars[row['Route_Key']] <= row['Max_Capacity_KT'], f"Cap_{row['Route_Key']}"

# US total supply constraint
model += lpSum([
    route_vars[r] for r in route_vars
]) <= 220, "US_Total_Supply"

# 5. Solve
model.solve()

# 6. Print results
print("Status:", LpStatus[model.status])
print("\nOptimal Shipment Plan (Scenario 2: France Capacity Boost):\n")
for key, var in route_vars.items():
    print(f"{key}: {var.varValue:.1f} KT")

print(f"\nTotal Profit: ${value(model.objective):,.2f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.11/dist-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/5eabef78f8f247c48943fa884e6f6d91-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/5eabef78f8f247c48943fa884e6f6d91-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 31 RHS
At line 39 BOUNDS
At line 40 ENDATA
Problem MODEL has 7 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 6 (0) columns and 6 (-6) elements
0  Obj -0 Dual inf 1984.3891 (6)
1  Obj 80190.448
Optimal - objective value 80190.448
After Postsolve, objective 80190.448, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80190.44757 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seco